In [1]:
import sys
import xlsxwriter
import pandas as pd
import sys
import psycopg2
import os
from dotenv import load_dotenv
from datetime import date

today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')

cnx = psycopg2.connect(
        user=DB_USERNAME,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        database=DB_DATABASE
    )
cursor=cnx.cursor()

In [4]:
data=pd.read_excel(r'V:\Archivos Seguimiento de Egresados\2025\2016\Bases 2016 teelefonos.xlsx',skiprows=[0])

In [5]:
data['cuenta']

0        52899784
1        67083510
2        70204269
3        73040361
4        73371430
          ...    
5802    414034235
5803    414043774
5804    414046074
5805    414058710
5806    414108080
Name: cuenta, Length: 5807, dtype: int64

In [9]:
eg16=pd.read_sql("select * from egresados where act_suvery='1'",cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [10]:
eg16['cuenta']

0       305227993
1       413094584
2       309148496
3       412087156
4       308187285
          ...    
5216     84093532
5217     95540225
5218     97551465
5219     77102159
5220     74260302
Name: cuenta, Length: 5221, dtype: object

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')

cnx = psycopg2.connect(
        user=DB_USERNAME,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        database=DB_DATABASE
    )

cursor=cnx.cursor()

In [ ]:
#SUBIR DATOS DE CONTACTO
# for i in range(len(data)):
#set interval:
strt=0
end=100
for i in range(strt,end):
    cuenta=str(int(data['cuenta'].values[i]))
    Eg=pd.read_sql("select * from egresados where cuenta='"+cuenta+"'",cnx)
    tels_exits=pd.read_sql("select * from telefonos where cuenta = '"+cuenta+"'",cnx )
    mails_exits=pd.read_sql("select * from correos where cuenta = '"+cuenta+"'",cnx )
    print(i,cuenta,Eg['nombre'].values[0],len(Eg))
    print(len(tels_exits),'telefonos')
    #incrustar a la base SQL los telefonos de la base en excel
    #telcasa
    if(len(data['telcasa'].values[i])>8):
        if(data['telcasa'].values[i][:3]=='044'):
            tel=data['telcasa'].values[i][3:]
        else:
            tel=data['telcasa'].values[i]
        if(tel not in tels_exits['telefono'].values):
            query=("insert into telefonos(cuenta, telefono,status,descripcion) values('%s','%s','%s','%s')"%tuple([cuenta,tel,0,'casa']))
            cursor.execute(query)
            print(tel,'insertado ::::TEL INSERTED::::')
    #telcel
    if(len(str(data['telcel'].values[i]))>8):
        print(data['telcel'].values[i])
        if(data['telcel'].values[i][:3]=='044'):
            tel=data['telcel'].values[i][3:]
        else:
            tel=data['telcel'].values[i]
        if(tel not in tels_exits['telefono'].values):
            query=("insert into telefonos(cuenta, telefono,status,descripcion) values('%s','%s','%s','%s')"%tuple([cuenta,tel,0,'celular']))
            cursor.execute(query)
    #correo
    if((len(str(data['nar1_a'].values[i]))>6)&('@' in str(data['nar1_a'].values[i]) )):
        if(data['nar1_a'].values[i][:1]=='-'):
            correo=data['nar1_a'].values[i][1:]
        else:
            correo=data['nar1_a'].values[i]
        if(correo not in mails_exits['correo'].values):
            query=("insert into correos(cuenta, correo,status) values('%s','%s','%s')"%tuple([cuenta,correo,0]))
            cursor.execute(query)
            print(correo,'insertado ::::MAIL INSERTED::::')
cnx.commit()

4001 310127778 DANIEL DE JESUS 1
0 telefonos
5534675855
danielcayetanolaurrabaquio@gmail.com insertado ::::MAIL INSERTED::::
4002 310128469 CAROLINA 1
0 telefonos
5539846594 insertado ::::TEL INSERTED::::
caro_89_allison@hotmail.com insertado ::::MAIL INSERTED::::
4003 310129071 INGRID TONANTZIN 1
0 telefonos
5587971125 insertado ::::TEL INSERTED::::
ingridnb2b@gmail.com insertado ::::MAIL INSERTED::::
4004 310131418 CLAUDIA ALEJANDRA 1
0 telefonos
5512498986
clausrubra1094@hotmail.com insertado ::::MAIL INSERTED::::
4005 310132075 DIEGO 1
0 telefonos
5525129577 insertado ::::TEL INSERTED::::
diegotg93@gmail.com insertado ::::MAIL INSERTED::::
4006 310132879 CARMEN YUDIET 1
0 telefonos
yudiet93@hotmail.com insertado ::::MAIL INSERTED::::
4007 310134213 ALBERTO 1
0 telefonos
5511350197 insertado ::::TEL INSERTED::::
larenasrr.ii@gmail.com insertado ::::MAIL INSERTED::::
4008 310136248 NADIA 1
0 telefonos
nadiacaro_2008rosa@hotmail.com insertado ::::MAIL INSERTED::::
4009 310138589 DIANA

In [98]:
#MARCAR LA FECHA DE LA ENCUESTA ANTERIOR
# for i in range(len(data)):
#set interval:
strt=2501
end=len(data)
for i in range(strt,end):
    cuenta=str(int(data['cuenta'].values[i]))
    Eg=pd.read_sql("select * from egresados where cuenta='"+cuenta+"'",cnx)
    print(i,cuenta,Eg['nombre'].values[0],len(Eg))
    if(len(Eg)>=1):
        query="update egresados set actualized='"+str(data['fec_capt'].values[i])+"' where cuenta='"+cuenta+"'"
        print(query)
        cursor.execute(query)

cnx.commit()

2501 308615058 RODRIGO 1
update egresados set actualized='2017-03-27T00:00:00.000000000' where cuenta='308615058'
2502 308615137 ANDRES 1
update egresados set actualized='2019-03-11T00:00:00.000000000' where cuenta='308615137'
2503 308618509 LILIA 1
update egresados set actualized='2017-02-02T00:00:00.000000000' where cuenta='308618509'
2504 308619197 DANIELA 1
update egresados set actualized='2020-03-19T00:00:00.000000000' where cuenta='308619197'
2505 308622256 ROSA LAURA 1
update egresados set actualized='2019-04-02T00:00:00.000000000' where cuenta='308622256'
2506 308624810 MARIANA 1
update egresados set actualized='2019-06-23T00:00:00.000000000' where cuenta='308624810'
2507 308625707 JOSE PABLO 1
update egresados set actualized='2019-06-05T00:00:00.000000000' where cuenta='308625707'
2508 308626436 KARINA 1
update egresados set actualized='2020-11-13T00:00:00.000000000' where cuenta='308626436'
2509 308633445 CARLOS EMILIO 1
update egresados set actualized='2019-06-05T00:00:00.00

In [90]:
data['fec_capt']

0      2019-08-19
1      2018-12-01
2      2020-12-10
3      2019-04-04
4      2020-06-10
          ...    
5802   2019-08-28
5803   2019-08-26
5804   2019-03-11
5805   2019-03-12
5806   2019-08-30
Name: fec_capt, Length: 5807, dtype: datetime64[ns]

In [ ]:
cnx.commit()

In [91]:
eg16['actualized']

0       None
1       None
2       None
3       None
4       None
        ... 
5216    None
5217    None
5218    None
5219    None
5220    None
Name: actualized, Length: 5221, dtype: object

In [53]:
data.telcasa=data.telcasa.astype(str)

In [54]:
data['telcel'].values[i]

'5540780218'

In [35]:
("insert into telefonos(cuanta, telefono,status,descripcion) values('%s','%s','%s','%s')"%tuple([cuenta,tel,0,'casa']))

"insert into telefonos(cuanta, telefono,status,descripcion) values('305588212','5525114200','0','casa')"

In [29]:
if(data['telcasa'].values[12][:3]=='044'):
   tel=data['telcasa'].values[12][3:]
print(tel)

5525114200


In [11]:
len(data)

5807

In [85]:
cnx.close()

In [36]:
tels_exits=pd.read_sql("select * from telefonos where cuenta = '123123333'",cnx )

In [39]:
'5567123456' not in tels_exits['telefono'].values

False

In [4]:
#Importar la columan de si estan titulados o no
import pandas as pd
data=pd.read_spss(r'V:\Archivos Seguimiento de Egresados\Bases de datos finales todos los años\EX2016_columnas solo 2016 final 17-10-23.sav')

In [6]:
data['nfr27']

0                Sí
1       En trámites
2                No
3       En trámites
4                Sí
           ...     
5215             No
5216             No
5217             No
5218             No
5219             No
Name: nfr27, Length: 5220, dtype: category
Categories (3, object): ['En trámites', 'No', 'Sí']

In [ ]:
#revisar poder encontrar todos los numeros de cuenta
for i in  range(len(data)):
    n=len(pd.read_sql("select * from egresados where cuenta ='"+str(int(data['cuenta'].values[i]))+"' and act_suvery=1",cnx))
    if(n!=1):
        print('atipic',n,str(int(data['cuenta'].values[i])))
       


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

KeyboardInterrupt: 

In [ ]:
import sys
import xlsxwriter
import pandas as pd
import sys
import psycopg2
import os
from dotenv import load_dotenv
from datetime import date

today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')
cnx = psycopg2.connect(
        user=DB_USERNAME,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        database=DB_DATABASE
    )
cursor=cnx.cursor()
   

In [21]:
#Hacer el update
for i in  range(len(data)):

# for i in  range(100):
    tit_val='0'
    if(data['nfr27'].values[i]=='En trámites'):
        tit_val='1'
    if(data['nfr27'].values[i]=='Sí'):
        tit_val='2'
    query="""UPDATE egresados set titulado= '"""+tit_val+"""' 
            where cuenta ='"""+str(int(data['cuenta'].values[i]))+"""' and act_suvery=1"""
    # print(query)
    # print(data['nfr27'].values[i])
    cursor.execute(query) 

In [22]:
cnx.commit()